In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import norm, sem
from utility import bisection_method

pd.options.display.float_format = '{:.4f}'.format

### 7.H
The setup is similar to exercise 6.F. Let $X$ and $Y$ be the prices of two underlying stocks.
Assume that they are both geometric Brownian motions under the risk-neutral probability measure and
$$ X_T = X_0 \exp\left((r - \frac{1}{2}\sigma_1^2)T + \sigma_1 \sqrt{T} Z_1\right),$$
$$Y_T = Y_0 \exp\left((r - \frac{1}{2}\sigma_2^2)T + \sigma_2 \sqrt{T} Z_2\right), $$
where $Z_1$ and $Z_2$ are two independent standard normal random variables.
We wish to estimate the price of a basket call option with maturity $T$ and payoff
$$ h(Z_1, Z_2) = (c_1 X_T + c_2 Y_T - K)^+ $$
by importance sampling. The alternative sampling distribution is assumed to be
$N(\theta, I_2)$ for some $\theta = (\theta_1, \theta_2) \in \mathbb{R}^2$.

(a) The mode matching method sets the tilting parameter $(θ_1,θ_2)$ as the
maximizer of $h(z_1, z_2)f(z_1, z_2)$, where $f$ is the joint density function of
$Z = (Z1, Z2)$. It is equivalent to maximizing
$$ h(z_1, z_2) \exp \left\{ -\frac{1}{2} (z_1^2 + z_2^2) \right\}.$$
Find the maximizer $(z_1^*, z_2^*)$ by the bisection method. Hint: Taking partial derivatives over $z_1$ and $z_2$ leads to two equations with two un-
knowns. Reduce the system of equations to a single equation with one
unknown.

(b) Write a function to estimate the call option price by impor-
tance sampling. The function should have input parameters $r$, $σ_1$, $σ_2$,
$X_0$, $Y_0$, $T$, $K$, $c_1$, $c_2$, and sample size $n$. Report your estimate, standard
error, and the tilting parameter $(θ_1,θ_2)$ for
$$r = 0.1, σ_1 = 0.2, σ_2 = 0.3, X_0 = Y_0 = 50, T = 1, c_1 = c_2 = 0.5,$$
and strike $K = 80, 100, 120$, respectively. Sample size is $n = 10000$.

In [8]:
def basket_call_option(r, sig1, sig2, X_0, Y_0, T, K, c1, c2, n):

    def plain(num_samples):
        Z1 = np.random.normal(0, 1, num_samples)
        Z2 = np.random.normal(0, 1, num_samples)
        
        X_T = X_0 * np.exp((r - 0.5 * sig1**2) * T + sig1 * np.sqrt(T) * Z1)
        Y_T = Y_0 * np.exp((r - 0.5 * sig2**2) * T + sig2 * np.sqrt(T) * Z2)
        
        payoffs = np.maximum(c1 * X_T + c2 * Y_T - K, 0)
        discount_factor = np.exp(-r * T)
        H = discount_factor * payoffs

        est = np.mean(H)
        se = sem(H)
        re = se / est if est != 0 else np.inf

        theta = np.array([0., 0.])

        return est, se, re, theta
    
    def mode_matching():
        def get_z2(z1):
            X_T = X_0 * np.exp((r - 0.5 * sig1**2) * T + sig1 * np.sqrt(T) * z1)
            temp = c1 * X_T * (sig1 * np.sqrt(T) / z1 - 1) + K
            temp = temp / (c2 * Y_0)
            
            z2 = (np.log(temp) - (r - 0.5 * sig2**2) * T) / (sig2 * np.sqrt(T))
            return z2


        def h(z1):
            X_T = X_0 * np.exp((r - 0.5 * sig1**2) * T + sig1 * np.sqrt(T) * z1)
            
            z2 = get_z2(z1)
            Y_T = Y_0 * np.exp((r - 0.5 * sig2**2) * T + sig2 * np.sqrt(T) * z2)
            
            payoff = c1 * X_T + c2 * Y_T - K

            return c2 * sig2 * np.sqrt(T) * Y_T - z2 * payoff
        
        z1_opt = bisection_method(h, 0.005, 5)
        z2_opt = get_z2(z1_opt)

        theta = np.array([z1_opt, z2_opt])
        return theta
    
    def importance_sampling(num_samples, theta):
        theta1, theta2 = theta

        Z1 = np.random.normal(theta1, 1, num_samples)
        Z2 = np.random.normal(theta2, 1, num_samples)

        X_T = X_0 * np.exp((r - 0.5 * sig1**2) * T + sig1 * np.sqrt(T) * Z1)
        Y_T = Y_0 * np.exp((r - 0.5 * sig2**2) * T + sig2 * np.sqrt(T) * Z2)

        payoffs = np.maximum(c1 * X_T + c2 * Y_T - K, 0)
        discount_factor = np.exp(-r * T)

        likelihood_ratio = np.exp(-theta1 * Z1 - theta2 * Z2 + 0.5 * (theta1**2 + theta2**2))

        H = discount_factor * payoffs * likelihood_ratio

        est = np.mean(H)
        se = sem(H)
        re = se / est if est != 0 else np.inf

        return est, se, re, theta

    theta = mode_matching()
    est_plain, se_plain, re_plain, theta_plain = plain(n)
    est_is, se_is, re_is, theta_is = importance_sampling(n, theta)

    re_plain = f"{re_plain * 100:.2f}%"
    re_is = f"{re_is * 100:.2f}%"

    theta_plain = f"({theta_plain[0]:.2f}, {theta_plain[1]:.2f})"
    theta_is = f"({theta_is[0]:.2f}, {theta_is[1]:.2f})"

    # put into dataframe
    results = pd.DataFrame({
        'Method': ['Plain MC', 'IS'],
        'Est.': [est_plain, est_is],
        'SE': [se_plain, se_is],
        'RE': [re_plain, re_is],
        'Theta': [theta_plain, theta_is]
    })

    return results
    

In [9]:
# Simulation parameters
r = 0.1
sig1, sig2 = 0.2, 0.3
X_0, Y_0 = 50, 50
T = 1
c1, c2 = 0.5, 0.5

K = [80, 100, 120]
n = 10_000

for k in K:
    res = basket_call_option(r, sig1, sig2, X_0, Y_0, T, k, c1, c2, n)
    display(res)

,Method,Est.,SE,RE,Theta
0,Plain MC,0.1129,0.0118,10.47%,"(0.00, 0.00)"
1,IS,0.1009,0.0011,1.06%,"(0.97, 2.38)"


,Method,Est.,SE,RE,Theta
0,Plain MC,0.0036,0.0016,45.29%,"(0.00, 0.00)"
1,IS,0.0037,0.0000,1.33%,"(1.03, 3.40)"


,Method,Est.,SE,RE,Theta
0,Plain MC,0.0000,0.0000,inf%,"(0.00, 0.00)"
1,IS,0.0001,0.0000,1.47%,"(0.99, 4.22)"
